In [3]:
import os
from PIL import *
import pandas as pd

In [ ]:
folder_path = "/Volumes/Extreme SSD/眼位写真NEW"
csv_path = "/Volumes/Extreme SSD/Disease_extracted_甲状腺眼症.バセドウ病眼症.csv"


def openCSV(csv_path):
    #df = pd.read_csv(csv_path, encoding="shift-jis")
    df = pd.read_csv(csv_path, encoding="utf-8", header=None)
    #df = df.drop(df.columns[[0]], axis=1) #1列目を削除する
    #print(df)
    return df

df = openCSV(csv_path)
df

In [51]:
#甲状腺眼症から始まるものを抽出
df_temp = df.dropna(subset=[8])
df_GO = df_temp[df_temp[8].str.startswith('甲状腺眼症')]

#中等症以上ものを抽出
df_temp = df.dropna(subset=[8])
df_moderate = df_temp[df_temp[8].str.contains('中等症')]

#ステロイドありを抽出
df_temp = df.dropna(subset=[9])
df_steroids = df_temp[df_temp[9].str.contains('ステロイドあり')]

#中等症以上とステロイドありを合算
df_treatable = pd.concat([df_moderate, df_steroids]).drop_duplicates()


GO_list = df_treatable[2].tolist()
GO_list

[17,
 19,
 57,
 122,
 129,
 140,
 155,
 180,
 374,
 385,
 412,
 425,
 426,
 429,
 466,
 489,
 507,
 533,
 534,
 555,
 611,
 718,
 724,
 782,
 787,
 818,
 876,
 893,
 950,
 957,
 965,
 977,
 983,
 1001,
 1037,
 1088,
 1231,
 1270,
 1284,
 1313,
 1333,
 1336,
 1347,
 1357,
 1370,
 1378,
 1385,
 1429,
 1432,
 1438,
 1446,
 1453,
 1457,
 1501,
 1503,
 1524,
 1525,
 1528,
 1553,
 1597,
 1664,
 1682,
 1729,
 1743,
 1764,
 1786,
 1815,
 1841,
 1910,
 1913,
 1931,
 2007,
 2042,
 2148,
 2182,
 2189,
 2203,
 2222,
 2231,
 2274,
 2307,
 2321,
 2370,
 2407,
 2414,
 2436,
 2508,
 2510,
 2558,
 2623,
 2630,
 2644,
 2658,
 2664,
 2668,
 2670,
 2684,
 2689,
 2709,
 2726,
 2762,
 2794,
 2801,
 2803,
 2837,
 2851,
 2853,
 2885,
 2900,
 2960,
 2967,
 2974,
 3046,
 3056,
 3077,
 3128,
 3144,
 3162,
 3164,
 3167,
 3209,
 3217,
 3237,
 3264,
 3276,
 3331,
 3352,
 3412,
 3444,
 3472,
 3484,
 3502,
 3528,
 3532,
 3543,
 3567,
 3568,
 3609,
 3646,
 3667,
 3695,
 3714,
 3748,
 3752,
 3783,
 3802,
 3843,
 3875,


In [56]:
os.path.basename(glob.glob(folder_path +"/*")[0])

'1-20141126-38-091804_eb568e2ac952f8be45ec0ac9ae800120b7c988b60ac499ca87306986d218f554.jpg'

In [71]:
import os
import glob
import shutil
from tqdm import tqdm


k=1
for path in tqdm(glob.glob(folder_path +"/*")):
    #画像IDがGO_listから始まるものを抜き出す
    #print(os.path.basename(path).rsplit("-")[0])
    if int(os.path.basename(path).rsplit("-")[0]) in GO_list:
        shutil.copy(path, f"/Volumes/Extreme SSD/Treatable_GO_images/{os.path.basename(path)}")
        k+=1

print(f"total {k} files picked up!")


100%|██████████| 79905/79905 [13:54<00:00, 95.74it/s]  

total 4432 files picked up!
